## TRATAR DE GENERAR VECTOR EMBEDDINGS DE LAS IMAGENES USANDO CLIP

In [2]:
import os
import torch
import clip
from PIL import Image
import pickle
from tqdm import tqdm  # For progress bar

In [3]:

external_drive_path = "/Volumes/Crucial X9 Pro For Mac/TP_Vision_Final"



In [5]:
import os
import csv
import pickle
from PIL import Image
from tqdm import tqdm
import torch
import clip
import pandas as pd  # For handling CSV files

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Paths
data_folder = "../data"  
image_folder = external_drive_path+"/sample_imgs" 

# CSV file names
csv_files = {
    "train": os.path.join(data_folder, "X_train_paths.csv"),
    #"test": os.path.join(data_folder, "X_test_paths.csv"),
    "val": os.path.join(data_folder, "X_val_paths.csv"),
}

# Function to process images and save features to a CSV file
def process_images(csv_file, output_file):
    # Load image filenames from the CSV
    image_names = pd.read_csv(csv_file, header=None).iloc[:, 0].tolist()

    # Initialize a list to store feature vectors
    feature_vectors = []

    # Process images with a progress bar
    for image_name in tqdm(image_names, desc=f"Processing {os.path.basename(csv_file)}", unit="image"):
        # Construct the full image path
        image_path = os.path.join(image_folder, image_name)

        if not os.path.exists(image_path):
            print(f"Warning: Image {image_name} not found in {image_folder}. Skipping.")
            feature_vectors.append([None] * 512)  # Optional: Fill with placeholder for missing images
            continue

        # Load and preprocess the image
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

        # Extract features
        with torch.no_grad():
            image_features = model.encode_image(image).cpu().numpy().flatten()  # Flatten the tensor

        # Add the features to the list
        feature_vectors.append(image_features)

    # Save the features as a CSV file
    pd.DataFrame(feature_vectors).to_csv(output_file, index=False, header=False)
    print(f"Feature vectors saved to {output_file}")

# Process each partition
for partition, csv_file in csv_files.items():
    output_file = f"../data/{partition}_clip_features.csv"
    process_images(csv_file, output_file)


Processing X_train_paths.csv:   0%|          | 3/64801 [00:00<59:49, 18.05image/s]

Processing X_train_paths.csv: 100%|██████████| 64801/64801 [1:48:50<00:00,  9.92image/s]  


Feature vectors saved to ../data/train_clip_features.csv


Processing X_val_paths.csv:   0%|          | 0/7201 [00:00<?, ?image/s]

Processing X_val_paths.csv: 100%|██████████| 7201/7201 [14:27<00:00,  8.30image/s]


Feature vectors saved to ../data/val_clip_features.csv


In [6]:
image_features.shape


torch.Size([1, 512])

In [4]:
text_features.shape

torch.Size([3, 512])